# MCP Server

## Introduction

In this notebook, we will demonstrate the use of the MCP server workflow.
This workflow runs a server that provides access to an ApertureDB instance via the Model Context Protocol (MCP).

## Setup

In order to run this notebook, you will need to be running the MCP server workflow.
You can do this conveniently in the Cloud UI.
You will also need to know the authorization bearer token for the MCP server.


In [1]:
from fastmcp import Client
from getpass import getpass
import os
import json

## Enter Authorization Bearer Token

In [2]:
TOKEN = getpass("authorization bearer token")

## Set up the client

The FastMCP library is commonly used to create MCP servers,
but here we use the client feature.

In [ ]:
URL = 'https://<DB_HOST>/mcp/'
client = Client(URL, auth=TOKEN)

## Test the connection

If everything is set up correctly, this will print `True`.

In [4]:
async with client:
    print(await client.ping())

True


## List tools

Let's find out what tools the server has.

In [12]:
async with client:
    tools = await client.list_tools()
    for tool in tools:
        print(f"Tool: {tool.name} - {tool.description}")
        print(f"  Parameters: {json.dumps(tool.inputSchema, indent=2, ensure_ascii=False)}")
        print()

Tool: list_entity_classes - List all entity classes in the database.
  Parameters: {
  "properties": {},
  "type": "object"
}

Tool: describe_entity_class - Describe an entity class in the database.
  Parameters: {
  "properties": {
    "class_name": {
      "description": "The name of the entity class to describe",
      "title": "Class Name",
      "type": "string"
    }
  },
  "required": [
    "class_name"
  ],
  "type": "object"
}

Tool: list_connection_classes - List all connection classes in the database.
  Parameters: {
  "properties": {},
  "type": "object"
}

Tool: describe_connection_class - Describe an connection class in the database.
  Parameters: {
  "properties": {
    "class_name": {
      "description": "The name of the connection class to describe",
      "title": "Class Name",
      "type": "string"
    }
  },
  "required": [
    "class_name"
  ],
  "type": "object"
}

Tool: find_similar_documents - Find documents that are similar to a given text string
  Parameters

## Run a tool: Find Similar Documents

Here we run the `find_similar_documents` tool against the default descriptor set and print the result documents.

In [ ]:
text = "Find documents similar to this text."
async with client:
    docs = await client.call_tool("find_similar_documents", dict(query=text))
    for doc in docs:
        print(json.dumps(json.loads(doc.text), indent=2, ensure_ascii=False))


## Run a tool: List Entity Classes

What types of entity are there?

In [ ]:
async with client:
    results = await client.call_tool("list_entity_classes")
    print(json.dumps(json.loads(results.content[0].text), indent=2, ensure_ascii=False))

## Run a tool: Count images

How many images are there in the database? We can see the `matched` field here.

In [ ]:
entity_class = "_Image"
async with client:
    description = await client.call_tool('describe_entity_class',
        dict(class_name=entity_class))
    print(json.dumps(json.loads(description.content[0].text), indent=2, ensure_ascii=False))